<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Entrenamiento/Entrenamiento_Detectron2_Varios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Conectar Colab a Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.data.datasets import register_coco_instances

In [ ]:
from ultralytics import YOLO
from matplotlib import pyplot as plt
from PIL import Image
import optuna
import locale
import os
import json
import pandas as pd
import yaml
import torch
import cv2
import random

# Verificación de CUDA
print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Número de dispositivos CUDA disponibles:", torch.cuda.device_count())
    print("Nombre del dispositivo CUDA actual:", torch.cuda.get_device_name())

# Hiperparámetros ajustados para YOLO
HYPERPARAMETERS = {
    'epochs': 10000,
    'patience': 250,
    'batch': [8, 32],
    'workers': [8, 16],
    'imgsz': [320, 1024],
    'save_period': -1,
    'num_trials': 5,
    'lr0': [1e-5, 1e-2],
    'momentum': [0.7, 0.99],
    'weight_decay': [1e-5, 1e-2],
    'warmup_epochs': [100, 1000],
    'warmup_momentum': [0.0, 0.95],
    'warmup_bias_lr': [1e-5, 1e-2],
    'optimizer': ['RMSProp', 'Adam', 'SGD'],
    'testing_threshold': [0.4, 0.6]
}

# Combinaciones de datasets y pesos preentrenados
combinaciones = [
    {"ruta_resultados": "models/Final/YoloV8_m_20_fix", "archivo_yaml": "/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/Polipos265_Detectron2FIX_YoloV8/data.yaml", "pre_weights": "yolov8m-seg.yaml"},
    {"ruta_resultados": "models/Final/YoloV8_m_20_pre", "archivo_yaml": "/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/DataAugmentation/Only_Preprocessing/data.yaml", "pre_weights": "yolov8m-seg.yaml"},
    {"ruta_resultados": "models/Final/YoloV8_m_20_all", "archivo_yaml": "/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/DataAugmentation/All_Data_Augmentation/data.yaml", "pre_weights": "yolov8m-seg.yaml"}
]

# Función objetivo para Optuna
def objective(trial, ruta_resultados, archivo_yaml, pre_weights):
    lr0 = trial.suggest_float('lr0', HYPERPARAMETERS['lr0'][0], HYPERPARAMETERS['lr0'][1], log=True)
    momentum = trial.suggest_float('momentum', HYPERPARAMETERS['momentum'][0], HYPERPARAMETERS['momentum'][1])
    weight_decay = trial.suggest_float('weight_decay', HYPERPARAMETERS['weight_decay'][0], HYPERPARAMETERS['weight_decay'][1], log=True)
    warmup_epochs = trial.suggest_int('warmup_epochs', HYPERPARAMETERS['warmup_epochs'][0], HYPERPARAMETERS['warmup_epochs'][1])
    warmup_momentum = trial.suggest_float('warmup_momentum', HYPERPARAMETERS['warmup_momentum'][0], HYPERPARAMETERS['warmup_momentum'][1])
    warmup_bias_lr = trial.suggest_float('warmup_bias_lr', HYPERPARAMETERS['warmup_bias_lr'][0], HYPERPARAMETERS['warmup_bias_lr'][1], log=True)
    optimizer = trial.suggest_categorical('optimizer', HYPERPARAMETERS['optimizer'])
    testing_threshold = trial.suggest_float('testing_threshold', HYPERPARAMETERS['testing_threshold'][0], HYPERPARAMETERS['testing_threshold'][1])

    batch_size = trial.suggest_int('batch', HYPERPARAMETERS['batch'][0], HYPERPARAMETERS['batch'][1], step=8)
    workers = trial.suggest_int('workers', HYPERPARAMETERS['workers'][0], HYPERPARAMETERS['workers'][1])
    imgsz = trial.suggest_int('imgsz', HYPERPARAMETERS['imgsz'][0], HYPERPARAMETERS['imgsz'][1], step=64)

    # Crear una instancia del modelo
    model = YOLO(pre_weights)

    # Configurar los parámetros de entrenamiento
    training_params = {
        'data': archivo_yaml,
        'project': ruta_resultados,
        'name': f"trial_{trial.number}",
        'epochs': HYPERPARAMETERS['epochs'],
        'patience': HYPERPARAMETERS['patience'],
        'batch': batch_size,
        'imgsz': imgsz,
        'lr0': lr0,
        'momentum': momentum,
        'weight_decay': weight_decay,
        'warmup_epochs': warmup_epochs,
        'warmup_momentum': warmup_momentum,
        'warmup_bias_lr': warmup_bias_lr,
        'optimizer': optimizer,
        'workers': workers,
        'save_period': HYPERPARAMETERS['save_period']
    }

    # Entrenar el modelo con los hiperparámetros sugeridos
    results = model.train(**training_params)

    # Definir la ruta para la carpeta del trial
    trial_dir = os.path.join(ruta_resultados, f"trial_{trial.number}")
    if not os.path.exists(trial_dir):
        os.makedirs(trial_dir)

    # Definir la ruta para la carpeta del trial y el archivo all_metrics.json
    model_path = os.path.join(trial_dir, "weights/best.pt")

    # Cargamos el modelo que se acaba de entrenar
    model_segmentation = YOLO(model_path)

    # Evaluar el modelo en el conjunto de test con el umbral de confianza especificado
    val_params = {
        'data': archivo_yaml,
        'conf': testing_threshold,
        'save_json': True,
        'split': "test",
        'project': trial_dir,
        'name': "Evaluacion"
    }
    print("Antes de validar")
    test_results = model_segmentation.val(**val_params)
    print("Despues de validar")

    # Obtener las métricas del modelo
    metrics = test_results.results_dict

    # Crear un DataFrame con las métricas
    df = pd.DataFrame([metrics])

    # Extraer y convertir las métricas
    precision = df["metrics/precision(M)"].iloc[0]
    recall = df["metrics/recall(M)"].iloc[0]
    mAP50 = df["metrics/mAP50(M)"].iloc[0]

    # Imprimir las métricas para verificar
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"mAP: {mAP50}")

    # Crear un diccionario con las métricas relevantes
    metrics_row = {
        "Trial": trial.number,
        "Dataset": archivo_yaml,
        "Feedback": "mAP50",
        "Pre-Trained_Weights": pre_weights,
        "metrics": {
            'precision': metrics.get('metrics/precision(M)', 0),
            'recall': metrics.get('metrics/recall(M)', 0),
            'mAP50': metrics.get('metrics/mAP50(M)', 0),
            'mAP50-95': metrics.get('metrics/mAP50-95(M)', 0),
            'fitness': metrics.get('fitness', 0)
        },
        "hyperparameters": {
            'lr0': lr0,
            'epochs': HYPERPARAMETERS['epochs'],
            'batch_size': batch_size,
            'imgsz': imgsz,
            'momentum': momentum,
            'weight_decay': weight_decay,
            'warmup_epochs': warmup_epochs,
            'warmup_momentum': warmup_momentum,
            'warmup_bias_lr': warmup_bias_lr,
            'optimizer': optimizer,
            'workers': workers,
            'testing_threshold': testing_threshold
        }
    }
    metrics_path = os.path.join(trial_dir, "metrics.json")
    with open(metrics_path, 'w') as outfile:
        json.dump(metrics_row, outfile)

    return mAP50

# Bucle para ejecutar el entrenamiento y la optimización para cada combinación de rutas y pesos
for combinacion in combinaciones:
    ruta_resultados = combinacion["ruta_resultados"]
    archivo_yaml = combinacion["archivo_yaml"]
    pre_weights = combinacion["pre_weights"]

    # Ejecutar la optimización con Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, ruta_resultados, archivo_yaml, pre_weights), n_trials=HYPERPARAMETERS['num_trials'])